# IBM Applied Data Science Capstone Course by Coursera
## Week 3 Part 2

### Add Latitude and Longitude to each Postal Code

### **Step 1:** Load Required Libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

### Step 2: Scrape the Wikipedia Page with Beautiful Soup

In [5]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source,'lxml')

### Step 3: Create New Lists to store the Table Data, then find and append the correct table data into each list

In [6]:
# Create lists to store the table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [7]:
# Append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

### Step 4: Create the DataFrame

In [8]:
# Create a new DataFrame using the three lists just populated
TorontoNeighborhoods = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

TorontoNeighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Step 5: Drop Boroughs that are "Not Assigned"

In [9]:
# Drop cells with a borough that is Not assigned
TorontoNeighborhoods_update1 = TorontoNeighborhoods[TorontoNeighborhoods.Borough != "Not assigned"].reset_index(drop=True)
TorontoNeighborhoods_update1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Step 6: Group Neighborhoods with the same Postal Code

In [10]:
# group neighborhoods in the same postal code
TorontoNeighborhoods_cleaned = TorontoNeighborhoods_update1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
TorontoNeighborhoods_cleaned.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Step 7: Populate any "Not assigned" Neighborhood with its Borough

In [11]:
# for any Neighborhood that is "Not assigned" make the value the same as Borough
for index, row in TorontoNeighborhoods_cleaned.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
TorontoNeighborhoods_cleaned.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Step 8: Confirm the Shape of the dataframe

In [12]:
TorontoNeighborhoods_cleaned.shape

(103, 3)

## Part 2: Integrating Latitude and Longitude

### Step 1: Load the coordinates from the CSV file

In [22]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

body = client_46a9ae20c285475db76c79b5cf7cb800.get_object(Bucket='capstoneproject-donotdelete-pr-hmpri1goikdkry',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

coordinates = pd.read_csv(body)
coordinates.head()



,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the two tables

In [20]:

# merge two table on the column "PostalCode"
TorontoNeighborhoods_LatLong = TorontoNeighborhoods_cleaned.merge(coordinates, on="PostalCode", how="left")
TorontoNeighborhoods_LatLong.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Step 2: Confirm the Shape of the new dataframe

In [21]:
TorontoNeighborhoods_LatLong.shape

(103, 5)